Decide on formulas for scaled ingredient quantities.

Formulas

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
plt.rcParams['figure.figsize'] = [12, 6]

In [83]:
# Load a sample of the DataFrame
#  Load only the columns relevant for scaling
core_cols = ['efficiency', 'boil_size', 'batch_size', 'boil_time', 'style_name']
ferm_cols = ['ferm_amount', 'ferm_yield']
hop_cols = ['hop_amount', 'hop_alpha', 'hop_form', 'hop_use']
num_recipes = 50000

store = pd.HDFStore("all_recipes.h5","r")

core = store.select("/core", columns=core_cols, where='index<{}'.format(num_recipes))

In [56]:
ferm = store.select("/ingredients", columns=ferm_cols, where='index<{}'.format(num_recipes))
ferm = ferm.join(core).dropna(subset=['ferm_amount'])

In [86]:
hop = store.select("/ingredients", columns=hop_cols, where='index<{}'.format(num_recipes))
hop = hop.join(core).dropna(subset=['hop_amount'])

In [81]:
def ferm_scaled(ferm_row): 
    """(Series) -> float
    Compute the scaled fermentable quantity.
    
    Take as input a row from the ing DataFrame, joined to the core DataFrame.
    Return the gravity contribution of the fermentable: 
        g/L extract in the boil kettle. """
    
    r = ferm_row
    f = r.ferm_amount * r.ferm_yield * r.efficiency / r.boil_size
    return f

In [82]:
ferm['ferm_scaled'] = ferm.apply(lambda x: ferm_scaled(x), axis=1)

In [98]:
def hop_scaled(hop_row):
    """(Series) -> float
    Compute the scaled hop quantity.
    
    Take as input a row from the ing DataFrame, joined to the core DataFrame.
    Return a different quantity depending on the use: 
        Dry hops:  dry hopping rate
            grams of dry hops per litre in the batch
        Boil hops: AUU
            grams of alpha acids per litre in the boil kettle"""
    
    r = hop_row 
    if r.hop_use == 'dry hop':
        h = r.hop_amount / r.batch_size
    else: 
        is_leaf = int(r.hop_form == "leaf")
        h = r.hop_amount * r.hop_alpha * (1 - 0.1 * is_leaf) / r.boil_size
    return h     

In [99]:
hop['hop_scaled'] = hop.apply(lambda x: hop_scaled(x), axis=1)

Investigations

In [116]:
ferm_scaled_totals = ferm['ferm_scaled'].groupby(by=ferm.index).sum()

In [130]:
# Calculate intuitive ferm totals for the median homebrew beer: lb per batch
ferm_homebrew = ferm_scaled_totals * core.boil_size.median() / (core.efficiency.median() * ferm.ferm_yield.median() * 0.454) 

In [141]:
dry_hop = hop[hop.hop_use == 'dry hop']
dry_hop_scaled_totals = dry_hop['hop_scaled'].groupby(by=dry_hop.index).sum()

In [157]:
# Calculate intuitive dry hop totals for the median homebrew beer: oz per batch
dry_hop_homebrew = dry_hop_scaled_totals * core.batch_size.median() * 35.3

In [161]:
boil_hop = hop[hop.hop_use != 'dry hop']
boil_hop_scaled_totals = boil_hop['hop_scaled'].groupby(by=boil_hop.index).sum()

In [162]:
# Calculate intuitive kettle hop totals for the median homebrew beer: pellet oz per batch, 5% AA 
boil_hop_homebrew = boil_hop_scaled_totals * core.boil_size.median() * 35.3 / (0.05)

In [163]:
boil_hop_homebrew

id
0         3.216656
1         3.052252
2         2.132253
3         5.183825
4         0.500369
5         4.373852
6        12.634309
7         0.650480
8         7.880807
9         0.880045
10        4.128804
11        1.902626
12        2.021489
13        8.545074
14        3.833740
15        3.527978
16        6.072182
17       16.525616
18        3.252400
19        0.833948
20        6.167309
21        0.460193
22        1.981462
23       16.972524
24        1.366138
25        1.780777
26        2.787531
27        1.242505
28        3.583369
29        2.361743
           ...    
49968     4.378226
49969     5.332709
49970     4.369321
49971     2.403734
49972    16.110497
49973     3.083258
49974     2.341725
49975     5.282259
49976     2.251659
49978     5.251547
49979    14.532197
49980     0.981828
49981     4.425135
49982     6.574851
49983     2.401772
49984     4.433194
49986     0.900665
49987     0.794008
49988    10.008469
49989     5.235865
49990     0.953828
49991    

In [165]:
ing.loc[6]

,ferm_amount,ferm_color,ferm_display_amount,ferm_name,ferm_origin,ferm_potential,ferm_yield,hop_alpha,hop_amount,hop_display_amount,...,misc_time,misc_use,yeast_amount,yeast_attenuation,yeast_flocculation,yeast_form,yeast_laboratory,yeast_name,yeast_product_id,yeast_type
id,,,,,,,,,,,,,,,,,,,,,
6,2.494756,4.0,5.5 lb,light lme,united kingdom,1.037,0.798964,0.150,0.028349,1.0 oz,...,NaN,NaN,NaN,76.5,NaN,liquid,white labs,california ale yeast,NaN,ale
6,1.814368,3.0,4.0 lb,maris otter,united kingdom,1.035,0.755776,0.130,0.028349,1.0 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.226796,1.0,0.5 lb,carapils® malt,united states,1.034,0.734183,0.105,0.028349,1.0 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.226796,120.0,0.5 lb,crystal 120,canada,1.032,0.690995,0.120,0.028349,1.0 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.226796,22.0,0.5 lb,aromatic barley malt,united kingdom,1.035,0.755776,0.120,0.028349,1.0 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.113398,50.0,0.25 lb,special roast,NaN,1.033,0.712589,0.150,0.028349,1.0 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130,0.028349,1.0 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
